# Table of content
## 1. Setting up
## 2. Solving given tasks
## Edit Lesson 4.9: handling detected outliers

# 1. Setting up

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# creating path for later import/export

path = r"C:\Users\Anwender\Documents\07-2023 Instacart Basket Analysis\02 Data"

In [4]:
# importing df

ords_prods_merge = pd.read_pickle(os.path.join(path, "Prepared Data", "df_derived_2.pkl"))

In [4]:
# checking shape and columns

ords_prods_merge.head()

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders


In [5]:
ords_prods_merge.shape

(32404859, 18)

# 2. Solving given tasks

## Step 2

In [6]:
# single aggregation of amount of orders over department_id

ords_prods_merge.groupby("department_id").agg({"amount_of_orders": ["mean"]})

,amount_of_orders
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### The results of the whole dataframe differ to the subset in the sense that the average amount of orders are lower compared to the subset. This is due to the fact that the subset only had a fraction of the values analyzed, which is why the values differ.

### Step 4

In [7]:
#Creating loyalty flag using loc

ords_prods_merge["max_order"] = ords_prods_merge.groupby(["user_id"])["amount_of_orders"].transform(np.max)

In [8]:
ords_prods_merge.head()

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10


In [9]:
# creating loyalty_flag for max_order:

ords_prods_merge.loc[ords_prods_merge["max_order"] > 40, "loyalty_flag"] = "Loyal customer"

In [10]:
ords_prods_merge.loc[(ords_prods_merge["max_order"] <= 40) & (ords_prods_merge ["max_order"] > 10), "loyalty_flag"] = "Regular customer"

In [11]:
ords_prods_merge.loc[ords_prods_merge["max_order"] <= 10, "loyalty_flag"] = "New customer"

In [12]:
# checking output

ords_prods_merge["loyalty_flag"].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

### Step 5

In [13]:
# checking the basic statistics of the product prices for each loyalty category

ords_prods_merge.groupby("loyalty_flag").agg({"prices": ["mean", "min", "max"]})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

#### There is barely a difference between new customers and regular customers' spending. Loyal customers tend to spend less money than the other customer groups.

### Step 6

In [14]:
# grouping average product prices of customers to create spending_flag

ords_prods_merge["average_spending"] = ords_prods_merge.groupby(["user_id"])["prices"].transform(np.mean)

In [15]:
# applying flag options

ords_prods_merge.loc[ords_prods_merge["average_spending"] < 10, "spending_flag"] = "Low spender"

In [16]:
ords_prods_merge.loc[ords_prods_merge["average_spending"] >= 10, "spending_flag"] = "High spender"

In [17]:
# checking output

ords_prods_merge.head(10)

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender


In [18]:
# checking for accuracy

ords_prods_merge.groupby("user_id")["prices"].mean()

user_id
1         6.367797
2         7.515897
3         8.197727
4         8.205556
5         9.189189
            ...   
206205    8.909375
206206    7.646667
206207    7.313453
206208    8.366617
206209    7.058915
Name: prices, Length: 206209, dtype: float64

In [19]:
# checking frequency count

ords_prods_merge["spending_flag"].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

### Step 7

In [20]:
# creating a frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

ords_prods_merge["median_days_previous_order"] = ords_prods_merge.groupby(["user_id"])["days_since_prior_order"].transform(np.median)

In [21]:
# checking output

ords_prods_merge.head(10)

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,median_days_previous_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,both,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5


In [22]:
# applying loc function to order_frequency flag:

ords_prods_merge.loc[ords_prods_merge["median_days_previous_order"] > 20, "order_frequency"] = "Non-frequent customer"

In [23]:
ords_prods_merge.loc[(ords_prods_merge["median_days_previous_order"] <= 20) & (ords_prods_merge["median_days_previous_order"] > 10), "order_frequency"] = "Regular customer"

In [24]:
ords_prods_merge.loc[ords_prods_merge["median_days_previous_order"] <= 10, "order_frequency"] = "Frequent customer"

In [25]:
# checking output

ords_prods_merge.head(20)

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,median_days_previous_order,order_frequency
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [26]:
ords_prods_merge["order_frequency"].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency, dtype: int64

#### Found missing value in order_frequency table.

In [27]:
# isolating NaN values:

df_null = ords_prods_merge[ords_prods_merge["order_frequency"].isnull() == True]

In [28]:
df_null.head()

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,median_days_previous_order,order_frequency
13645692,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17251990,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17622767,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
24138593,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
25880002,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN


In [31]:
df_null.shape

(5, 24)

#### The table above shows a user, who only ever made one order and hasn't returned for another ever since.

### Dropping column "busiest_day" as it was supposed to be substituted with "busiest_days"

In [29]:
ords_prods_merge = ords_prods_merge.drop(columns = ["busiest_day"])

In [30]:
ords_prods_merge.head()

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,median_days_previous_order,order_frequency
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [32]:
ords_prods_merge.shape

(32404859, 23)

### Step 9

In [8]:
# exporting final df

ords_prods_merge.to_pickle(os.path.join(path, "Prepared Data", "df_derived_2.pkl"))

# Edit Lesson 4.9 : handling detected outliers

In [5]:
# find prices over 100 (as unusual for supermarket prices)

ords_prods_merge.loc[ords_prods_merge["prices"] > 100]

,order_id,user_id,amount_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,median_days_previous_order,order_frequency
10030345,912404,17,12,2,14,5.0,21553,5,0,Lowfat 2% Milkfat Cottage Cheese,...,both,High-range product,Regularly busy,Most orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
10030346,603376,17,22,6,16,4.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,...,both,High-range product,Regularly busy,Average orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
10030347,3264360,135,2,2,21,13.0,21553,6,0,Lowfat 2% Milkfat Cottage Cheese,...,both,High-range product,Regularly busy,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
10030348,892534,135,3,0,8,12.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,...,both,High-range product,Busiest days,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
10030349,229704,342,8,1,19,30.0,21553,9,0,Lowfat 2% Milkfat Cottage Cheese,...,both,High-range product,Busiest days,Average orders,16,Regular customer,114.426619,High spender,23.0,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29166209,2249946,204099,29,0,8,4.0,33664,1,0,2 % Reduced Fat Milk,...,both,High-range product,Busiest days,Average orders,39,Regular customer,1106.743956,High spender,4.0,Frequent customer
29166210,2363282,204099,31,0,9,2.0,33664,1,1,2 % Reduced Fat Milk,...,both,High-range product,Busiest days,Average orders,39,Regular customer,1106.743956,High spender,4.0,Frequent customer
29166211,3181945,204395,13,3,15,8.0,33664,25,0,2 % Reduced Fat Milk,...,both,High-range product,Least busy,Most orders,15,Regular customer,451.153540,High spender,5.0,Frequent customer
29166212,2486215,205227,7,3,20,4.0,33664,8,0,2 % Reduced Fat Milk,...,both,High-range product,Least busy,Average orders,12,Regular customer,1178.381871,High spender,12.0,Regular customer


In [6]:
# replace all those values with "nan"

ords_prods_merge.loc[ords_prods_merge["prices"] >100, "prices"] = np.nan

In [7]:
# checking whether max value changed from 9999.00

ords_prods_merge["prices"].max()

25.0